<a href="https://colab.research.google.com/github/MicaelaRomeroC3/COPs/blob/main/04_Tablaporgestor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
ruta_historical = '/content/drive/MyDrive/COPS/DATA/3.HISTORICAL/llamadas.parquet'
ruta_salida = '/content/drive/MyDrive/COPS/OUTPUT/TABLA_GESTOR.xlsx'
os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
if os.path.exists(ruta_historical):
    df = pd.read_parquet(ruta_historical)
    print(f"📊 Histórico cargado con {len(df)} registros.")
else:
    print("⚠️ No se encontró el archivo histórico.")
    exit()

📊 Histórico cargado con 10000 registros.


In [ ]:
# === Variables de cada tipo ===
columnas_eval_1 = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'S1', 'S2', 'S3', 'S4', 'S5M', 'C1M', 'C2M', 'T1M', 'T2M', 'T3']
columnas_eval_2 = [
    'P11', 'P12', 'P13', 'P14', 'P15', 'P21', 'P22', 'P23',
    'P31', 'P32', 'P33', 'S11', 'S12', 'S13',
    'S31', 'S32', 'S33', 'S34', 'S41', 'S42', 'S43', 'S44', 'S45', 'S46',
    'S51', 'S52', 'C11', 'C12', 'C13', 'C21', 'C22',
    'T11', 'T12', 'T13', 'T14', 'T21', 'T22',
    'T31', 'T32', 'T33', 'T34', 'T35', 'T36', 'T37'
]

# === Ponderaciones ===
ponderacion_dict = {
    'P1': 0.02, 'P2': 0.10, 'P3': 0.03, 'P4': 0.03, 'P5': 0.05, 'P6': 0.02,
    'S1': 0.03, 'S2': 0.02, 'S3': 0.10, 'S4': 0.10, 'S5M': 0.05,
    'C1M': 0.10, 'C2M': 0.10, 'T1M': 0.05, 'T2M': 0.10, 'T3': 0.10
}

# === Tabla transpuesta 1 ===
errores = df.groupby(['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME'])[columnas_eval_1].apply(
    lambda x: (x == False).sum()
).reset_index()

llamadas = df.groupby(['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME']).size().reset_index(name='NUM_LLAMADAS')

resultado_1 = pd.merge(errores, llamadas, on=['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME'])

resultado_transpuesta1 = resultado_1.melt(
    id_vars=['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME', 'NUM_LLAMADAS'],
    value_vars=columnas_eval_1,
    var_name='VARIABLE',
    value_name='N_FALSOS'
)

resultado_transpuesta1['%_CUMPLIMIENTO'] = (
    (1 - resultado_transpuesta1['N_FALSOS'] / resultado_transpuesta1['NUM_LLAMADAS']) * 100
).round(2)

resultado_transpuesta1['%_CUMPLIMIENTO_TEXTO'] = resultado_transpuesta1['%_CUMPLIMIENTO'].astype(str) + '%'

# === Tabla transpuesta 2 ===
agrupado = df.groupby(['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME'])
n_verdaderos = agrupado[columnas_eval_2].apply(lambda x: (x == True).sum()).reset_index()

resultado_transpuesta2 = n_verdaderos.melt(
    id_vars=['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME'],
    value_vars=columnas_eval_2,
    var_name='VARIABLE',
    value_name='N_TRUE'
)

# === Relación padre-subvariable ===
relacion_vars = {
    'P1': ['P11', 'P12', 'P13', 'P14', 'P15'],
    'P2': ['P21', 'P22', 'P23'],
    'P3': ['P31', 'P32', 'P33'],
    'S1': ['S11', 'S12', 'S13'],
    'S3': ['S31', 'S32', 'S33', 'S34'],
    'S4': ['S41', 'S42', 'S43', 'S44', 'S45', 'S46'],
    'S5M': ['S51', 'S52'],
    'C1M': ['C11', 'C12', 'C13'],
    'C2M': ['C21', 'C22'],
    'T1M': ['T11', 'T12', 'T13', 'T14'],
    'T2M': ['T21', 'T22'],
    'T3': ['T31', 'T32', 'T33', 'T34', 'T35', 'T36', 'T37']
}
mapa_inverso = {subvar: padre for padre, subvars in relacion_vars.items() for subvar in subvars}
resultado_transpuesta2['VARIABLE_PADRE'] = resultado_transpuesta2['VARIABLE'].map(mapa_inverso)

agrupado_verdaderos = resultado_transpuesta2.groupby(
    ['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME', 'VARIABLE_PADRE']
)['N_TRUE'].sum().reset_index().rename(columns={'VARIABLE_PADRE': 'VARIABLE'})

# === Unir tablas ===
tabla_unificada = pd.merge(
    resultado_transpuesta1,
    agrupado_verdaderos,
    on=['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME', 'VARIABLE'],
    how='left'
)

tabla_unificada['N_TRUE'] = tabla_unificada['N_TRUE'].fillna(0).astype(int)

# === Calcular cumplimiento ponderado por variable ===
resultado_transpuesta1['PONDERACION'] = resultado_transpuesta1['VARIABLE'].map(ponderacion_dict)
resultado_transpuesta1['%_CUMPLIMIENTO_PONDERADO'] = (
    resultado_transpuesta1['%_CUMPLIMIENTO'] * resultado_transpuesta1['PONDERACION']
).round(4)

# === Calcular cumplimiento total (sumaproducto) ===
cumplimiento_total = resultado_transpuesta1.copy()
cumplimiento_total = cumplimiento_total.groupby(['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME']).apply(
    lambda x: (x['%_CUMPLIMIENTO'] * x['PONDERACION']).sum()
).reset_index(name='%_CUMPLIMIENTO_TOTAL')

# === Crear hoja dashboard ===
dashboard = cumplimiento_total.copy()
dashboard['%_CUMPLIMIENTO_TOTAL'] = dashboard['%_CUMPLIMIENTO_TOTAL'].round(2)
dashboard['%_CUMPLIMIENTO_TOTAL_TEXTO'] = dashboard['%_CUMPLIMIENTO_TOTAL'].astype(str) + '%'

Exception ignored in: <function ZipFile.__del__ at 0x7e9dd5f25f80>
Traceback (most recent call last):
  File "/usr/lib/python3.12/zipfile/__init__.py", line 1966, in __del__
    self.close()
  File "/usr/lib/python3.12/zipfile/__init__.py", line 1983, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file
/tmp/ipython-input-3058866192.py:92: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  cumplimiento_total = cumplimiento_total.groupby(['EMPRESA', 'AGENTE', 'FECHA_EVALUACIÓN', 'FECHA_INFORME']).apply(


In [ ]:
# === Guardar Excel con las 4 pestañas ===
with pd.ExcelWriter(ruta_salida, engine='openpyxl', mode='w') as writer:
    resultado_transpuesta1.to_excel(writer, sheet_name='AUXILIAR1', index=False)
    resultado_transpuesta2.to_excel(writer, sheet_name='AUXILIAR2', index=False)
    tabla_unificada.to_excel(writer, sheet_name='TABLA_UNIFICADAGEST', index=False)
    dashboard.to_excel(writer, sheet_name='DASHBOARD', index=False)

print(f"✅ Excel generado correctamente con las 4 pestañas en: {ruta_salida}")

✅ Excel generado correctamente con las 4 pestañas en: /content/drive/MyDrive/COPS/OUTPUT/TABLA_GESTOR.xlsx
